# Rule: **prepare_network**

**Outputs**

- resources/networks/`elec_s{simpl}_{clusters}_ec_l{ll}_{opts}.nc`


In [ ]:
######################################## Parameters

### Run
name = 'co2_05'
prefix = 'add'

### Network
simpl = ''
clusters = 100
ll = 'v1.0'
opts = ''

In [ ]:
##### Import packages
import pypsa
import pandas as pd
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import yaml
import os 
import sys
import numpy as np


##### Import local functions
sys.path.append(os.path.abspath(os.path.join('..')))
import functions as xp


##### Read params.yaml
with open('../params.yaml', 'r') as configfile:
    params = yaml.safe_load(configfile)


##### Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


##### Region files
file_regions_onshore = f'regions_onshore_elec_s{simpl}_{clusters}.geojson'
file_regions_offshore = f'regions_offshore_elec_s{simpl}_{clusters}.geojson'
path_regions = f'{params['rootpath']}/resources/{prefix}/{name}/'
gdf_regions_onshore = gpd.read_file(path_regions+file_regions_onshore)
gdf_regions_offshore = gpd.read_file(path_regions+file_regions_offshore)


##### NUTS files (must contain at least columns 'NUTS_ID' and 'geometry')
file_NUTS2 = 'NUTS2_ES.geojson'
file_NUTS3 = 'NUTS3_ES.geojson'
path_NUTS = f'{params['rootpath']}/data_ES/nuts/'
gdf_NUTS2 = gpd.read_file(path_NUTS+file_NUTS2)
gdf_NUTS3 = gpd.read_file(path_NUTS+file_NUTS3)


## `elec_s{simpl}_{clusters}_ec_l{ll}_{opts}.nc`

Load the network and show its components.

In [ ]:
file = f'elec_s{simpl}_{clusters}_ec_l{ll}_{opts}.nc'
path = f'{params['rootpath']}/resources/{prefix}/{name}/networks/'

n = pypsa.Network(path+file)

n

Plot the network.

In [ ]:
#################### Parameters
line_widths = 1*n.lines.s_nom / 1e3
link_widths = 1*n.links.p_nom / 1e3



#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, line_widths=line_widths, link_widths=link_widths, bus_sizes=params['bus_sizes'], bus_colors=params['bus_colors'], boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Variable: `n.global_constraints`

Place `n.global_constraints` in a dataFrame and show its content.

In [ ]:
gc = n.global_constraints

gc

### Variable: `n.lines`

Changes observed:
- Colum `capital_cost` has been modified (<<< CHECK POTENTIAL BUG, see function **set_transmission_limit** ,there is an update of line costs, that does not include config.yaml:lines:length_factor (1.25).)
- Column `s_nom_max`has been filled.

Place `n.lines` in a dataFrame and show its content.

In [ ]:
ln = n.lines

ln.head()

How is the relationship between line capital costs and line length?

In [ ]:
#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ln.plot(ax=ax, kind='scatter', x='length', y='capital_cost')

ax.set_xlabel('km')
ax.set_ylabel('EUR')
ax.grid(True, linestyle='--', alpha=0.5)

How is the relationship between s_nom and s_nom_max?

In [ ]:
#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ln.plot(ax=ax, kind='scatter', x='s_nom', y='s_nom_max', color='orange')

ax.set_xlabel('km')
ax.set_ylabel('EUR')
ax.grid(True, linestyle='--', alpha=0.5)

### Variable: `n.links`

Changes observed:
- Colum `capital_cost` has been modified (<<< CHECK POTENTIAL BUG)
- Column `s_nom_max`has been filled.

Place `n.links` in a dataFrame and show its content.

In [ ]:
lk = n.links

lk.head()